# `gpytorch`

Examples of using the `gpytorch` package for Gaussian Processes in PyTorch.
Documention for the package can be found [here](https://github.com/cornellius-gp/gpytorch).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Regression Tutorial

This tutorial is an example from the `gpytorch` documentation.
The original can be found [here](https://github.com/cornellius-gp/gpytorch/blob/master/examples/01_Simple_GP_Regression/Simple_GP_Regression.ipynb)

In this example, the features of `gpytorch` will be illustrated within the context of a simple regression problem.
Here we will train a Radial Basis Function (RBF) Kernel Gaussian Process to model

$$
\begin{align}
    y &= \sin(2 \pi x) + \varepsilon \\
    \varepsilon &\sim \mathcal{N}(0, 0.2)
\end{align}
$$

We will have $11$ training examples, and evaluate the performance on $51$ test examples.

In [2]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

ImportError: cannot import name '_standard_normal' from 'torch.distributions.utils' (/usr/local/lib/python3.7/site-packages/torch/distributions/utils.py)